In [14]:
colab = False

if colab:
    from google.colab import drive
    drive.mount('/content/drive')
    root = '/content/drive/MyDrive/colab/sat-images/'
    device = 'cuda'
    !wget -O sat.zip https://github.com/lacykaltgr/satellite-image-segmentation/archive/refs/heads/laci-420.zip
    !unzip sat.zip
    !cp -r satellite-image-segmentation-laci-420/src .
else:
    root = '/Users/laszlofreund/code/ai/satellite-image-segmentation/data/'
    device = 'mps'

In [15]:
import torch
from torch.utils.data import DataLoader
from src.dataset import XYDataset, load_dataset

x_train, y_train, x_val, y_val, x_test = load_dataset(root)

train_dataset = XYDataset(x_train, y_train, crop_size=512)
val_dataset = XYDataset(x_val, y_val, crop_size=512)
#test_dataset = XYDataset(x_test, torch.zeros(len(x_test)))

batch_size = 128
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
#test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [16]:
from src.model import UNet, EarlyStopper
unet = UNet().to(device)

In [ ]:
es = EarlyStopper(patience=10)
unet.train_model(train_loader, val_loader, es, num_epochs=1000, learning_rate=0.0001, device=device)

In [20]:
predictions, results = unet.predict(train_loader, device=device)

RuntimeError: Invalid buffer size: 8.00 GB

In [ ]:
eval = unet.evaluate(val_loader, device=device)

In [ ]:
from matplotlib.pyplot import imshow, show
imshow(predictions[0])
show()

In [ ]:
#TODO: confusion matrix

In [ ]:
predictions[0].shape

In [4]:
import torch

In [5]:
#TARGET
target = torch.tensor(train_dataset[0][1]).unsqueeze(0) # + BATCH_SIZE
target.shape

torch.Size([1, 9, 512, 512])

In [6]:
#OUTPUT
output = unet(train_dataset[0][0].unsqueeze(0).to(device))
output.shape

torch.Size([1, 9, 512, 512])

In [7]:
_, target_max = torch.max(target, 1)
target_max.shape

torch.Size([1, 512, 512])

In [8]:
_, output_max = torch.max(output, 1)
output_max.shape

torch.Size([1, 512, 512])

In [9]:
equal = output_max.to(device) == target_max.to(device)
equal.shape

torch.Size([1, 512, 512])

In [11]:
equal.sum()

tensor(22305, device='mps:0')

In [12]:
512*512

262144

In [18]:
unet.load_state_dict(torch.load('/Users/laszlofreund/code/ai/satellite-image-segmentation/unet.h5'))

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.